In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  

In [2]:
url = "https://realpython.github.io/fake-jobs/"
response = requests.get(url)

In [56]:
response.status_code

200

In [4]:
soup = BeautifulSoup(response.text)

In [45]:
# print(soup.prettify())

a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  

In [19]:
soup.find('h2', 'title')

<h2 class="title is-5">Senior Python Developer</h2>

In [20]:
type(soup.find('h2', 'title'))

bs4.element.Tag

In [21]:
soup.find('h2', 'title').text

'Senior Python Developer'

b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.  

In [26]:
job_title_tags = soup.find_all('h2','title')

In [28]:
job_titles = [tag.text.strip() for tag in job_title_tags]

In [59]:
# job_titles

c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  

In [60]:
job_cards = soup.find_all('div', 'card-content')

In [51]:
# job_cards 

In [61]:
titles = []
companies = []
locations = []
posting_dates = []

In [62]:
for card in job_cards:
    title = card.find('h2', class_='title').text.strip()
    company = card.find('h3', class_='company').text.strip()
    location = card.find('p', class_='location').text.strip()
    date = card.find('time')['datetime'].strip()
    
    titles.append(title)
    companies.append(company)
    locations.append(location)
    posting_dates.append(date)

In [63]:
for title, company, location, date in zip(titles, companies, locations, posting_dates):
    print(f"{title} | {company} | {location} | {date}")

Senior Python Developer | Payne, Roberts and Davis | Stewartbury, AA | 2021-04-08
Energy engineer | Vasquez-Davidson | Christopherville, AA | 2021-04-08
Legal executive | Jackson, Chambers and Levy | Port Ericaburgh, AA | 2021-04-08
Fitness centre manager | Savage-Bradley | East Seanview, AP | 2021-04-08
Product manager | Ramirez Inc | North Jamieview, AP | 2021-04-08
Medical technical officer | Rogers-Yates | Davidville, AP | 2021-04-08
Physiological scientist | Kramer-Klein | South Christopher, AE | 2021-04-08
Textile designer | Meyers-Johnson | Port Jonathan, AE | 2021-04-08
Television floor manager | Hughes-Williams | Osbornetown, AE | 2021-04-08
Waste management officer | Jones, Williams and Villa | Scotttown, AP | 2021-04-08
Software Engineer (Python) | Garcia PLC | Ericberg, AE | 2021-04-08
Interpreter | Gregory and Sons | Ramireztown, AE | 2021-04-08
Architect | Clark, Garcia and Sosa | Figueroaview, AA | 2021-04-08
Meteorologist | Bush PLC | Kelseystad, AA | 2021-04-08
Audiolo

d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [64]:
jobs_df = pd.DataFrame({
    'Title': titles,
    'Company': companies,
    'Location': locations,
    'Posting Date': posting_dates
})

print(jobs_df.head())

                     Title                     Company              Location  \
0  Senior Python Developer    Payne, Roberts and Davis       Stewartbury, AA   
1          Energy engineer            Vasquez-Davidson  Christopherville, AA   
2          Legal executive  Jackson, Chambers and Levy   Port Ericaburgh, AA   
3   Fitness centre manager              Savage-Bradley     East Seanview, AP   
4          Product manager                 Ramirez Inc   North Jamieview, AP   

  Posting Date  
0   2021-04-08  
1   2021-04-08  
2   2021-04-08  
3   2021-04-08  
4   2021-04-08  


2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.   

a. First, use the BeautifulSoup find_all method to extract the urls.  

In [67]:
apply_urls = []

for card in job_cards:
    apply_link = card.find_all('a')[-1]['href'].strip()
    apply_urls.append(apply_link)


print(jobs_df[['Title', 'Apply URL (Extracted)']].head())

                     Title                              Apply URL (Extracted)
0  Senior Python Developer  https://realpython.github.io/fake-jobs/jobs/se...
1          Energy engineer  https://realpython.github.io/fake-jobs/jobs/en...
2          Legal executive  https://realpython.github.io/fake-jobs/jobs/le...
3   Fitness centre manager  https://realpython.github.io/fake-jobs/jobs/fi...
4          Product manager  https://realpython.github.io/fake-jobs/jobs/pr...


In [68]:
print(jobs_df['Apply URL (Extracted)'].head(10).to_list())

['https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html', 'https://realpython.github.io/fake-jobs/jobs/energy-engineer-1.html', 'https://realpython.github.io/fake-jobs/jobs/legal-executive-2.html', 'https://realpython.github.io/fake-jobs/jobs/fitness-centre-manager-3.html', 'https://realpython.github.io/fake-jobs/jobs/product-manager-4.html', 'https://realpython.github.io/fake-jobs/jobs/medical-technical-officer-5.html', 'https://realpython.github.io/fake-jobs/jobs/physiological-scientist-6.html', 'https://realpython.github.io/fake-jobs/jobs/textile-designer-7.html', 'https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html', 'https://realpython.github.io/fake-jobs/jobs/waste-management-officer-9.html']


b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [70]:
import re

def slugify(title):
    slug = title.lower()
    slug = re.sub(r'[^\w\s-]', '', slug)
    slug = re.sub(r'\s+', '-', slug)
    return slug.strip('-')

base_url = "https://realpython.github.io/fake-jobs/jobs/"
manual_urls = [f"{base_url}{slugify(title)}-{i}.html" for i, title in enumerate(titles)]

jobs_df['Apply URL (Constructed)'] = manual_urls

In [72]:
# manual_urls

3. Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  

In [73]:
job_url = "https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html"

response = requests.get(job_url)
soup = BeautifulSoup(response.content, "html.parser")

job_desc_div = soup.find("div", class_="content")
job_description = job_desc_div.find("p").get_text(strip=True)

print(job_description)

Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.


b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  

In [74]:
def get_job_description(url):
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    job_desc_div = soup.find("div", class_="content")
    
    job_description = job_desc_div.find("p").get_text(strip=True)
    
    return job_description

In [76]:
url = "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html"
description = get_job_description(url)

print(description)

At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.


In [81]:
jobs_df["Description"] = jobs_df["Apply URL (Extracted)"].apply(get_job_description)

In [82]:
jobs_df

,Title,Company,Location,Posting Date,Apply URL (Extracted),Apply URL (Constructed),Description
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...,https://realpython.github.io/fake-jobs/jobs/se...,Professional asset web application environment...
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...,https://realpython.github.io/fake-jobs/jobs/en...,Party prevent live. Quickly candidate change a...
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/le...,https://realpython.github.io/fake-jobs/jobs/le...,Administration even relate head color. Staff b...
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fi...,https://realpython.github.io/fake-jobs/jobs/fi...,Tv program actually race tonight themselves tr...
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/pr...,https://realpython.github.io/fake-jobs/jobs/pr...,Traditional page a although for study anyone. ...
...,...,...,...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/mu...,https://realpython.github.io/fake-jobs/jobs/mu...,Paper age physical current note. There reality...
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/ra...,https://realpython.github.io/fake-jobs/jobs/ra...,Able such right culture. Wrong pick structure ...
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/da...,https://realpython.github.io/fake-jobs/jobs/da...,Create day party decade high clear. Past trade...
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fu...,https://realpython.github.io/fake-jobs/jobs/fu...,Pressure under rock next week. Recognize so re...
